This notebook has blocks of code that can be used for timing data access to the geopolar dataset. The code here has been run from multiple compute locations. The spreadsheet timing results is in google drive.

In [1]:
import xarray as xr
import dask.array as da
import zarr

# Summary Information

## AOI for Timing

AOI_A) 1 degree x 1 degree (`geopolar.sel(lat=slice(40, 41), lon=slice(-70, -69))`)
* 2853600 (2.85 million) `.size`, 11.4MB `.bytes`

AOI_B) 2 degree x 2 degree (`geopolar.sel(lat=slice(40, 42), lon=slice(-70, -68))`)
* `.size`, `.bytes`


## Results
AOI_A)
* loading data (`.values`): 12 min 50s (:gasp:)
* calculation (loop): 44s
* total: 13 min 34s

AOI_B)
* loading data (`.values`): 16 min 15s (2 min 30 sec CPU time)
* calculation (loop): not timed


## Projection
Estimate (linear) for full area (~500 square degrees):
* loading data (`.values`): 6417 minutes = 4.5 DAYS. DAYS.
* calculation (loop): 1 hour 53 minutes

Goodness.

### Writes - no dask
A) 0.1 degree x 0.2 degree (`geopolar.sel(lat=slice(40, 40.1), lon=slice(-70, -69.8))`)
* `.to_zarr()`: 13 minutes. For 0.2 square degrees :double-gasp:

... they were both about 13 minutes ... is it possible that is how long it takes to access every chunk? AKA if we scaled up to the full area it still may not take much longer than 13 minutes?

# Code Snippets for Timed Runs

## Accessing data with `xarray`

In [2]:
from dask.distributed import Client

Client()

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 16
Total threads: 128,Total memory: 503.40 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44272,Workers: 16
Dashboard: http://127.0.0.1:8787/status,Total threads: 128
Started: Just now,Total memory: 503.40 GiB
Comm: tcp://127.0.0.1:44353,Total threads: 8
Dashboard: http://127.0.0.1:33798/status,Memory: 31.46 GiB
Nanny: tcp://127.0.0.1:34010,


In [3]:
%%time

# open dataset
filepath = 'https://ncsa.osn.xsede.org/Pangeo/pangeo-forge/noaa-coastwatch-geopolar-sst-feedstock/noaa-coastwatch-geopolar-sst.zarr'
geopolar = xr.open_zarr(filepath)
# subset
geopolar = geopolar.analysed_sst.sel(lat=slice(40, 42), lon=slice(-70, -68))
# access data
sst = geopolar.values

CPU times: user 2min 16s, sys: 14.9 s, total: 2min 30s
Wall time: 16min 15s


## Timing `zarr` access

In [5]:
from dask.distributed import Client

Client()

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 16
Total threads: 128,Total memory: 503.40 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40610,Workers: 16
Dashboard: http://127.0.0.1:8787/status,Total threads: 128
Started: Just now,Total memory: 503.40 GiB
Comm: tcp://127.0.0.1:37245,Total threads: 8
Dashboard: http://127.0.0.1:36390/status,Memory: 31.46 GiB
Nanny: tcp://127.0.0.1:44123,


### 1 - using the `zarr` library

In [6]:
%%time

# open dataset
filepath = 'https://ncsa.osn.xsede.org/Pangeo/pangeo-forge/noaa-coastwatch-geopolar-sst-feedstock/noaa-coastwatch-geopolar-sst.zarr'
geopolar = zarr.open(filepath, mode='r')
sst = geopolar.analysed_sst

# subset and access data
data_subset = sst[:, 2600:2620, 2200:2220]

CPU times: user 11min 20s, sys: 5min 33s, total: 16min 53s
Wall time: 13min 48s


### 2 - using `da.from_zarr()`

In [10]:
%%time

# open dataset
filepath = 'https://ncsa.osn.xsede.org/Pangeo/pangeo-forge/noaa-coastwatch-geopolar-sst-feedstock/noaa-coastwatch-geopolar-sst.zarr'
geopolar = da.from_zarr(filepath, component='/analysed_sst')

# compute
sst_subset = geopolar[:, 2600:2620, 2200:2220]
sst_subset = sst_subset.compute()

CPU times: user 2min 53s, sys: 12.4 s, total: 3min 6s
Wall time: 12min 53s


### netcdf openmfdataset

In [4]:
xr.open_mfdataset('/data/pacific/rwegener/noaa-geopolar-nc/*.nc')

<xarray.Dataset>
Dimensions:           (time: 7134, lat: 3600, lon: 7200)
Coordinates:
  * time              (time) datetime64[ns] 2002-09-01T12:00:00 ... 2022-03-2...
  * lat               (lat) float32 -89.97 -89.93 -89.88 ... 89.88 89.93 89.97
  * lon               (lon) float32 -180.0 -179.9 -179.9 ... 179.9 179.9 180.0
Data variables:
    analysed_sst      (time, lat, lon) float32 dask.array<chunksize=(1, 3600, 7200), meta=np.ndarray>
    analysis_error    (time, lat, lon) float32 dask.array<chunksize=(1, 3600, 7200), meta=np.ndarray>
    sea_ice_fraction  (time, lat, lon) float32 dask.array<chunksize=(1, 3600, 7200), meta=np.ndarray>
    mask              (time, lat, lon) float32 dask.array<chunksize=(1, 3600, 7200), meta=np.ndarray>
Attributes: (12/47)
    Conventions:                CF-1.4, Unidata Observation Dataset v1.0
    title:                      Analysed blended sea surface temperature over...
    summary:                    An SST estimation scheme which combines multi...
    references:                 Fieguth,P.W. et al. "Mapping Mediterranean al...
    institution:                Office of Satellite Products and Operations
    comment:                    The Geo-Polar Blended Sea Surface Temperature...
    ...                         ...
    publisher_name:             The GHRSST Project Office
    publisher_url:              http://www.ghrsst.org
    publisher_email:            ghrsst-po@nceo.ac.uk
    processing_level:           L4
    cdm_data_type:              grid
    history:                    Fri Oct 19 11:12:34 2018: ncatted -a add_offs...